In [13]:
import numpy as np
from typing import Tuple, List
import time

In [14]:
class DataProcessor:
    """
    Uma classe avançada para processamento de dados que demonstra uso sofisticado de métodos NumPy em cenários práticos.
    """

    def __init__(self, shape: Tuple[int, int], seed: int = 42):
        """
        Inicializa o processador com um array de forma específica.

        Args:
            shape: Tupla definindo a forma do array (linhas, colunas)
            seed: Semente para reprodutibilidade (padrão: 42)
        """
        np.random.seed(seed)
        self.data = np.random.rand(*shape)
        self.original = self.data.copy()
        self.processing_history = []

    def sliding_window_processor(self, window_size: int = 3) -> np.ndarray:
        """
        Processa dados usando uma janela deslizante e demonstrao uso avançado de views para otimização de memória.
        """
        rows, cols = self.data.shape
        result = np.zeros((rows - window_size + 1, cols - window_size + 1))

        # Usando views para criar janelas deslizantes eficientes.
        for i in range(rows - window_size + 1):
            for j in range(cols - window_size + 1):
                window = self.data[i:i+window_size, j:j+window_size].view()
                result[i, j] = window.mean()

        return result

    def adaptive_thresholding(self, sensitivity: float = 1.0) -> np.ndarray:
        """
        Aplica limiarização adaptativa usando diferentes métodos NumPy.
        Demonstra uso avançado de copy() e fill().
        """
        result = self.data.copy()

        # Calcula estatísiticas locais
        mean = np.mean(result)
        std = np.std(result)

        # Cria máscara adaptiva
        mask = np.zeros_like(result)
        mask.fill(mean)

        # Aplica threshold adaptativo
        threshold = mask + (std * sensitivity)
        result[result < threshold] = 0

        return result

    def matrix_decompostion(self) -> Tuple[np.ndarray, List[float]]:
        """
        Realiza decomposição de matriz usando diferentes views.
        Demonstra uso avançado de tolist() e item().
        """
        eigenvalues, eigenvectors = np.linalg.eig(self.data)

        # Coverte eigenvalues para lista Python para processamento
        eig_list = eigenvalues.tolist()

        # Ordena os eigenvalue e mantém os índices
        sorted_indices = np.argsort(eigenvalues)[::-1]

        # Cria uma view dos eigenvectors ordenados
        sorted_vectors = eigenvectors[:, sorted_indices].view()

        # Extrai o primeiro eigenvalue como escalar
        principal_component = sorted_vectors[:, 0]
        principal_value = eigenvalues.item(sorted_indices[0])

        return principal_component, eig_list

    def process_with_memory_optimization(self, chunk_size: int=1000) -> np.ndarray:
        """
        Processa grandes conjuntos de dados em chunks para otimização de memória.
        Demonstra uso avançado de itemset() e view().
        """
        rows, cols = self.data.shape
        result = np.zeros_like(self.data)

        for i in range(0, rows, chunk_size):
            end_idx = min(i + chunk_size, rows)

            # Processa um chunk por vez usando view
            chunk = self.data[i:end_idx, :].view()
            processed_chunk = np.exp(-chunk * chunk)

            # Atualiza o resultado usando itemset para cada elemento
            for r in range(chunk.shape[0]):
                for c in range(cols):
                    result.itemset((i+r, c), processed_chunk[r, c])

        return result

    def benchmark_operations(self) -> dict:
        """
        Realiza o bechmark das diferentes operações e retorna métricas.
        """
        metrics = {}

        # Benchmark sliding window
        start_time = time.time()
        _ = self.sliding_window_processor()
        metrics['sliding_window'] = time.time() - start_time

        # Benchmark thresholding
        start = time.time()
        _ = self.adaptive_thresholding()
        metrics['thresholding'] = time.time() - start

        # Bechmark decompostion
        start = time.time()
        _ = self.matrix_decompostion()
        metrics['decomposition'] = time.time() - start

        return metrics

In [15]:
# Exemplo de uso
if __name__ == "__main__":
    # Cria processador com matriz 100x100
    processor = DataProcessor((100, 100))

    # Executa processamentos
    print("Processando dados...")

    # Aplica janela deslizante
    window_result = processor.sliding_window_processor()
    print(f'Forma do resultado da janela deslizante: {window_result.shape}')

    # Aplica thresholding adaptativo
    threshold_result = processor.adaptive_thresholding(sensitivity=1.2)
    print(f'Valores únicos após thresholding: {np.unique(threshold_result).size}')

    # Realiza decomposição
    principal_comp, eigenvalues = processor.matrix_decompostion()
    print(f'Primeiro autovalor: {eigenvalues[0]:.4f}')

    # Executa benchmark
    metrics = processor.benchmark_operations()
    for operation, time_taken, in metrics.items():
        print(f'{operation}: {time_taken:.4f} segundos')

Processando dados...
Forma do resultado da janela deslizante: (98, 98)
Valores únicos após thresholding: 1545
Primeiro autovalor: 49.4061+0.0000j
sliding_window: 0.0533 segundos
thresholding: 0.0002 segundos
decomposition: 0.0083 segundos
